In [1]:
import os
import re
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
import scanpy as sc
import torch
import torchvision.transforms.functional as TF
import tifffile
import yaml
from einops import rearrange


In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

In [4]:
from mushroom.data.multiplex import extract_ome_tiff, write_basic_ome_tiff, get_ome_tiff_channels, make_pseudo
from mushroom.data.visium import adata_from_visium
from mushroom.visualization.utils import display_sections
import mushroom.utils as utils

/data/estorrs/miniconda3/envs/mushroom/lib/python3.9/site-packages/geopandas/_compat.py:124: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/data/estorrs/miniconda3/envs/mushroom/lib/python3.9/site-packages/spatialdata/__init__.py:9: UserWarning: Geopandas was set to use PyGEOS, changing to shapely 2.0 with:

	geopandas.options.use_pygeos = True

If you intended to use PyGEOS, set the option to False.
  _check_geopandas_using_shapely()


In [6]:
run_dir = '/data/estorrs/mushroom/data/projects/kathleen_visium'
mushroom_dir = os.path.join(run_dir, 'mushroom')
Path(mushroom_dir).mkdir(parents=True, exist_ok=True)

In [7]:
metadata = yaml.safe_load(open(os.path.join(run_dir, 'registered', 'metadata.yaml')))
metadata

{'17B41236A': {'data': {'visium': {'17B41236A-A': '/diskmnt/Projects/Users/estorrs/mushroom/data/projects/kathleen_visium/registered/17B41236A/s0_visium.h5ad',
    '17B41236A-B': '/diskmnt/Projects/Users/estorrs/mushroom/data/projects/kathleen_visium/registered/17B41236A/s1_visium.h5ad',
    '17B41236A-C': '/diskmnt/Projects/Users/estorrs/mushroom/data/projects/kathleen_visium/registered/17B41236A/s2_visium.h5ad',
    '17B41236A-D': '/diskmnt/Projects/Users/estorrs/mushroom/data/projects/kathleen_visium/registered/17B41236A/s3_visium.h5ad'}},
  'ids': ['s0', 's1', 's2', 's3'],
  'order': ['17B41236A-A', '17B41236A-B', '17B41236A-C', '17B41236A-D'],
  'registered_pixels_per_micron': 1.1578182157391153,
  'scale': 0.2}}

In [8]:
configs = {}
for case, data in metadata.items():
    config = [
        {
            'id': sid,
            'position': 0,
            'data': []
        }
        for sid in data['ids']
    ]
    
    for dtype, mapping in data['data'].items():
        for sample_id, filepath in mapping.items():
            idx = data['order'].index(sample_id)
            
            # convert from katmai to glacier
            filepath = filepath.replace('/diskmnt/Projects/Users/estorrs', '/data/estorrs')
            
            config[idx]['data'].append(
                {
                    'dtype': dtype,
                    'filepath': filepath,
                }
            )
    configs[case] = config
configs
    

{'17B41236A': [{'id': 's0',
   'position': 0,
   'data': [{'dtype': 'visium',
     'filepath': '/data/estorrs/mushroom/data/projects/kathleen_visium/registered/17B41236A/s0_visium.h5ad'}]},
  {'id': 's1',
   'position': 0,
   'data': [{'dtype': 'visium',
     'filepath': '/data/estorrs/mushroom/data/projects/kathleen_visium/registered/17B41236A/s1_visium.h5ad'}]},
  {'id': 's2',
   'position': 0,
   'data': [{'dtype': 'visium',
     'filepath': '/data/estorrs/mushroom/data/projects/kathleen_visium/registered/17B41236A/s2_visium.h5ad'}]},
  {'id': 's3',
   'position': 0,
   'data': [{'dtype': 'visium',
     'filepath': '/data/estorrs/mushroom/data/projects/kathleen_visium/registered/17B41236A/s3_visium.h5ad'}]}]}

In [9]:
import sys
yaml.safe_dump(configs, sys.stdout)

17B41236A:
- data:
  - dtype: visium
    filepath: /data/estorrs/mushroom/data/projects/kathleen_visium/registered/17B41236A/s0_visium.h5ad
  id: s0
  position: 0
- data:
  - dtype: visium
    filepath: /data/estorrs/mushroom/data/projects/kathleen_visium/registered/17B41236A/s1_visium.h5ad
  id: s1
  position: 0
- data:
  - dtype: visium
    filepath: /data/estorrs/mushroom/data/projects/kathleen_visium/registered/17B41236A/s2_visium.h5ad
  id: s2
  position: 0
- data:
  - dtype: visium
    filepath: /data/estorrs/mushroom/data/projects/kathleen_visium/registered/17B41236A/s3_visium.h5ad
  id: s3
  position: 0


In [19]:
# # filling out position seperately and reading back in
# yaml.safe_dump(configs, open(os.path.join(mushroom_dir, 'configs_template.yaml'), 'w'))

In [10]:
configs = yaml.safe_load(open(os.path.join(mushroom_dir, 'configs_template.yaml')))
configs

{'17B41236A': [{'data': [{'dtype': 'visium',
     'filepath': '/data/estorrs/mushroom/data/projects/kathleen_visium/registered/17B41236A/s0_visium.h5ad'}],
   'id': 's0',
   'position': 0},
  {'data': [{'dtype': 'visium',
     'filepath': '/data/estorrs/mushroom/data/projects/kathleen_visium/registered/17B41236A/s1_visium.h5ad'}],
   'id': 's1',
   'position': 125},
  {'data': [{'dtype': 'visium',
     'filepath': '/data/estorrs/mushroom/data/projects/kathleen_visium/registered/17B41236A/s2_visium.h5ad'}],
   'id': 's2',
   'position': 250},
  {'data': [{'dtype': 'visium',
     'filepath': '/data/estorrs/mushroom/data/projects/kathleen_visium/registered/17B41236A/s3_visium.h5ad'}],
   'id': 's3',
   'position': 375}]}

In [11]:
for case, config in configs.items():
    directory = os.path.join(mushroom_dir, case)
    Path(directory).mkdir(parents=True, exist_ok=True)
    
    yaml.safe_dump(config, open(os.path.join(directory, 'config.yaml'), 'w'))

In [12]:
mushroom_dir

'/data/estorrs/mushroom/data/projects/kathleen_visium/mushroom'